## Imports ##

In [1]:
import numpy as np
import pandas as pd
import math
from numpy import random
from numpy import linalg

## Input ##

In [2]:
m_row = 1000 
n_col = 10000 # needs to be big
possibilities = [-1, 1]

In [3]:
def initialize(m_row, n_col):
    W = np.random.normal(size = (m_row, n_col)) 
    X = np.random.choice(possibilities, n_col)
    Y = np.maximum((np.dot(W, X)/math.sqrt(n_col)), 0)
    return W, X, Y

W, X, Y = initialize(m_row, n_col)
T = 1
B = 1/T
n_iter = 10
N = 100
print(np.shape(W), np.shape(X), np.shape(Y))

(1000, 10000) (10000,) (1000,)


## Helper functions ##

In [4]:
def energy(vector):
    _sum = 0
    dot_product = np.dot(W, vector)
    for i in range(len(W)):
        _sum += (Y[i] - np.maximum(0, dot_product[i]/math.sqrt(n_col)))**2
    return _sum

In [5]:
print(energy(X) == 0.0)

True


In [33]:
def error(vector):
    return np.sum((vector - X)**2) / (4 * n_col)

In [86]:
def Metropolis_chain(dim, n_iter, B, rate, rate_iter_change):
    test = np.random.choice(possibilities, dim)
    errors = []
    low_energy = 10000
    
    for _iter in range(n_iter):
        #if _iter > n_iter*treshold:
        if _iter % int(rate_iter_change * n_iter) == int(rate_iter_change * n_iter) - 1: 
            B = B * rate
            
        to_flip = np.random.randint(0, dim)
        to_test = np.copy(test)
        to_test[to_flip] = -to_test[to_flip]
        proba = np.minimum(1, np.exp(-B*(energy(to_test)-energy(test))))
        
        if random.random() < proba:
            # Accepted case
            test = to_test
            
        # Save new error term for plot
        errors.append(error(test))
        
        # Save lowest energy
        ener = energy(test)
        if ener < low_energy:
            low_energy = ener
    
    low_error = errors[-1]
    return test, low_energy, low_error, errors    

In [8]:
def Glauber(dim, n_iter, B, treshold, rate):
    test = np.random.choice(possibilities, dim)
    for _iter in range(n_iter):
        if _iter > n_iter*treshold:
            B = B*rate
        to_flip = np.random.randint(0, dim)
        proba = (1 + test[to_flip]*math.tanh(B*(energy(test * -1) - energy(test))))/2.0
        if random.random() < proba:
            test[to_flip] = 1 
        else:
            test[to_flip] = -1
    return test, error(test)

In [103]:
# Run of Metropolis algo to find best parameters:
"""dim = len(X)
inits_B = [0.1, 0.25, 0.5, 1, 5, 10] # different initialization for B
changes_B = [2, 5, 10, 20, 50, 75, 100, 150] # different number of iterations before increasing B

lowest_energy = 10000
best_errors = []
best_init_B = 0
best_change_B = 0

print('Start')
for init_B in inits_B:
    for change_B in changes_B:
        X_hat, ener, metropolis_errs = Metropolis_chain(dim, 1000, init_B, change_B)
        if ener < lowest_energy:
            lowest_energy = ener
            best_errors = metropolis_errs
            best_init_B = init_B
            best_change_B = change_B
        print("init_B: {} change_B: {} energy: {}".format(init_B, change_B, ener))"""

Start
init_B: 0.1 change_B: 2 energy: 444.4237458448838
init_B: 0.1 change_B: 5 energy: 477.3635012544742
init_B: 0.1 change_B: 10 energy: 433.6029711406104
init_B: 0.1 change_B: 20 energy: 446.6967689011138
init_B: 0.1 change_B: 50 energy: 418.161792832904
init_B: 0.1 change_B: 75 energy: 459.610228074645
init_B: 0.1 change_B: 100 energy: 465.94527043975125
init_B: 0.1 change_B: 150 energy: 504.085762193623
init_B: 0.25 change_B: 2 energy: 443.50944952185046
init_B: 0.25 change_B: 5 energy: 450.0673427670765
init_B: 0.25 change_B: 10 energy: 452.7454178680771
init_B: 0.25 change_B: 20 energy: 484.71130810511
init_B: 0.25 change_B: 50 energy: 443.1068253107573
init_B: 0.25 change_B: 75 energy: 505.50650375657057
init_B: 0.25 change_B: 100 energy: 473.5301659674238
init_B: 0.25 change_B: 150 energy: 460.188500442851
init_B: 0.5 change_B: 2 energy: 446.3301484573286
init_B: 0.5 change_B: 5 energy: 452.5969832071845
init_B: 0.5 change_B: 10 energy: 483.1361132700293
init_B: 0.5 change_B: 

## Optimization ##

In [83]:
_, low_energy, metro_error, errors = Metropolis_chain(n_col, 1000, B = 1, rate = 5, rate_iter_change = 0.05)

In [85]:
print(low_energy)
print(metro_error)
print(min(errors))

401.7608609587946
0.4891
0.4889


In [89]:
n_iter = 2000
B = 0.25
rates_change = np.linspace(0.05, 0.5, num=10)
t_increasing = np.linspace(1.2, 5, num=8)

errors_tab = np.zeros((len(t_increasing), len(rates_change)))

best_error = 1e6
best_set_up = None

for i, rate_change in enumerate(rates_change):
    for j, increasing in enumerate(t_increasing):
        for test in range(2):
            metro_results = []
            #glauber_results = []
            _, low_energy, metro_error, errors = Metropolis_chain(n_col, n_iter, B, increasing, rate_change)
            #glauber_error = Glauber(n_col, n_iter, B, treshold, increasing)
            metro_results.append(metro_error)
            #glauber_results.append(glauber_error)
            print("Rate change: {}, Rate increase beta: {}".format(rate_change, increasing))
            print("Lowest energy, last error, min error: {} , {} , {}".format(low_energy, metro_error, min(errors)))
        metro_mean = np.mean(metro_results)
        errors_tab[i, j] = metro_mean
        if(metro_mean < best_error):
            best_error = metro_mean
            best_set_up = ("Metropolis", "treshold :",treshold, "rate :", increasing, "error :", best_error)
        """glauber_mean = np.mean(glauber_results)
        if(glauber_mean < best_error):
            best_error = glauber_mean
            best_set_up = ("Glauber " ,"treshold :",treshold, "rate :", increasing, "error :", best_error)"""

KeyboardInterrupt: 

## Results ##  

In [ ]:
errors = []
for i i range(N):
    error = #Glauber ou Metropolis_chain selon best_set_up
    errors.append(error)
mean = np.mean(errors)
variance = np.var(errors)
print('Mean error : ', mean)
print('Variance of error : ' variance)

In [86]:
best_set_up

('Metropolis', 'treshold :', 0.5, 'rate :', 5.0, 'error :', 0.015)

In [82]:
t_increasing = np.linspace(1.5, 5, num=8)
t_increasing

array([1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])

In [ ]:
BEST_SO_FAR = 0.01411756195508118 #Metropolis_chain(n_col, 100, B) 0.8